![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/BQm_EV6i0_j80CQZ8vcLvw/SN-web-lightmode.png)


# **Lab: Creating Custom Layers and Models**

###### Estimated time needed:  30 minutes  

In this lab, you will learn to create custom layers and integrate them into a Keras model. You will compile, train, and evaluate the model. 

##### Learning objectives 

By the end of this lab, you will: 
- Create custom layers and integrate them into a Keras model 
- Compile, train, and evaluate the model 

##### Prerequisites:
- Basic understanding of Python and Keras. 


#### Steps 

**Step 1: Import libraries**

Before you start, import the required libraries: TensorFlow and Keras. Keras is included within TensorFlow as `tensorflow.keras`. 


In [2]:
!pip install tensorflow==2.16.2
import tensorflow as tf

from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Sequential

**Step 2: Define a custom layer**

Define a custom dense layer with 32 units and ReLU activation.


In [13]:
# Define a custom dense layer by subclassing the Layer class
class CustomDenseLayer(Layer):
    def __init__(self, units=32):
        # Initialize the parent class and store the number of units (neurons) for the layer
        super(CustomDenseLayer, self).__init__()
        self.units = units

    # The build method is called the first time the layer is used to define its weights
    def build(self, input_shape):
        # Define the weight matrix with shape (input features, units)
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',  # Initialize weights with a random normal distribution
                                 trainable=True)  # These weights are trainable

        # Define the bias vector with shape (units,)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='zeros',  # Initialize biases to 0
                                 trainable=True)  # These biases are also trainable

    # The call method is where the layer's forward pass is defined
    def call(self, inputs):
        # Perform matrix multiplication of the inputs and weights, then add the bias
        # Apply ReLU activation function to the result
        return tf.nn.relu(tf.matmul(inputs, self.w) + self.b)
        


**Step 3: Integrate the custom layer into a model**

Create a Keras model using the custom layer. 


In [14]:
from tensorflow.keras.layers import Softmax

# Define the model with a Softmax activation in the output layer
model = Sequential([
    CustomDenseLayer(128),   # Custom dense layer with 128 neurons and ReLU activation
    CustomDenseLayer(10),    # Custom dense layer with 10 neurons and ReLU activation
    Softmax()                # Softmax layer for multi-class classification
])


The **Softmax** activation function is used in the output layer for multi-class classification tasks, ensuring the model outputs probabilities that sum up to 1 for each class, which aligns with categorical cross-entropy as the loss function. This adjustment ensures the model is optimized correctly for multi-class classification.


**Step 4: Compile the model**

Compile the model with the Adam optimizer and categorical cross-entropy loss. 


In [15]:
# Compile the model with Adam optimizer and categorical cross-entropy loss
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Print model summary before building (weights and parameters are not yet initialized)
print("Model summary before building:")
model.summary()

# Build the model with a specific input shape
# - Shape: (1000, 20) means 1000 samples, each with 20 features
model.build((1000, 20))

# Print model summary after building (weights and parameters are now initialized)
print("\nModel summary after building:")
model.summary()


Model summary before building:


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ custom_dense_layer_4            │ ?                      │   0 (unbuilt) │
│ (CustomDenseLayer)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_dense_layer_5            │ ?                      │   0 (unbuilt) │
│ (CustomDenseLayer)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax_1 (Softmax)             │ ?                      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ custom_dense_layer_4            │ (1000, 128)            │         2,688 │
│ (CustomDenseLayer)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_dense_layer_5            │ (1000, 10)             │         1,290 │
│ (CustomDenseLayer)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ softmax_1 (Softmax)             │ (1000, 10)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,978 (15.54 KB)

 Trainable params: 3,978 (15.54 KB)

 Non-trainable params: 0 (0.00 B)

**Step 5: Train the model**

Train the model on some example data. For this example, you will generate random data for training. In practice, use a real data set. 


In [8]:
import numpy as np 

# Generate random input data
# - x_train: A dataset with 1000 samples, each containing 20 features
x_train = np.random.random((1000, 20)) 

# Generate random labels
# - y_train: 1000 labels, randomly assigned values from 0 to 9 (for 10 classes)
y_train = np.random.randint(10, size=(1000, 1)) 

# Convert labels to categorical one-hot encoding
# - y_train: Converts the integer labels (0–9) into one-hot encoded format
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10) 

# Train the model on the randomly generated data
# - epochs=10: Train for 10 full passes over the dataset
# - batch_size=32: Process the data in mini-batches of 32 samples each
model.fit(x_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2.3076 
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.3010
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.2962
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2945
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2948
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2896
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2903
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2864
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2828
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2793


**Step 6: Evaluate the model**

Evaluate the model using test data to see its performance. 

For this example, you will generate random test data. In practice, use a real data set. 


In [9]:
# Generate random test data
# - x_test: A dataset with 200 samples, each containing 20 features
x_test = np.random.random((200, 20)) 

# Generate random labels for the test data
# - y_test: 200 labels, randomly assigned values from 0 to 9 (for 10 classes)
y_test = np.random.randint(10, size=(200, 1)) 

# Convert labels to categorical one-hot encoding
# - y_test: Converts the integer labels (0–9) into one-hot encoded format
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10) 

# Evaluate the model's performance on the test data
# - Returns the loss and other metrics (e.g., accuracy, if specified during compilation)
loss = model.evaluate(x_test, y_test) 

# Print the test loss
print(f'Test loss: {loss}')


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 2.2956  
Test loss: 2.2925074100494385


### Exercises


#### Exercise 1: Visualize Model Architecture

**Objective:** Visualize the architecture of the custom Keras model to understand its structure.

**Instructions:**
1. Use the `plot_model` function from `tensorflow.keras.utils` to visualize the model architecture.
2. Save the plot as an image file.


In [10]:
# Install the necessary libraries for model visualization
!pip install pydot graphviz

# Import the `plot_model` utility from TensorFlow Keras
from tensorflow.keras.utils import plot_model

# Visualize the model architecture and save it to a file
# - `to_file`: Specifies the filename where the architecture image will be saved
# - `show_shapes`: If True, displays the shapes of the input and output tensors for each layer
# - `show_layer_names`: If True, displays the names of the layers
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 15.2 MB/s eta 0:00:00
You must install pydot (`pip install pydot`) for `plot_model` to work.


<details>
    <summary>Click here for Solution</summary>

```python

!pip install pydot graphviz

from tensorflow.keras.utils import plot_model

# Visualize the model architecture
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)


 ```   

</details>


#### Exercise 2: Add Dropout Layer

**Objective:** Enhance the model by adding a Dropout layer to prevent overfitting.

**Instructions:**
1. Add a Dropout layer between the custom dense layers.
2. Recompile the model and observe the impact on training.


In [11]:
from tensorflow.keras.layers import Dropout

# Modify the model to include a Dropout layer
# - Dropout: A regularization technique to reduce overfitting
# - CustomDenseLayer: Custom dense layers with ReLU activation
model = Sequential([
    CustomDenseLayer(64),    # First custom dense layer with 64 neurons
    Dropout(0.5),            # Dropout layer that randomly drops 50% of neurons during training
    CustomDenseLayer(10)     # Second custom dense layer with 10 neurons
])

# Recompile the model
# - Use Adam optimizer and categorical cross-entropy loss for multi-class classification
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Train the model again
# - epochs=10: Train the model for 10 epochs
# - batch_size=32: Divide the data into mini-batches of 32 samples each
model.fit(x_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 7.7688 
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.9697
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.7181
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.3929
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.3883
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.3548
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.4072
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.3319
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.3392
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.2986


<details>
    <summary>Click here for Solution</summary>

```python

from tensorflow.keras.layers import Dropout

# Modify the model to include a Dropout layer
model = Sequential([
    CustomDenseLayer(64),
    Dropout(0.5),
    CustomDenseLayer(10)
])

# Recompile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Train the model again
model.fit(x_train, y_train, epochs=10, batch_size=32)
 ```   

</details>


#### Exercise 3: Adjust the Number of Units in Custom Layer

**Objective:** Experiment with different numbers of units in the custom dense layer to observe the impact on performance.

**Instructions:**
1. Change the number of units in the `CustomDenseLayer` to 128.
2. Recompile, train, and evaluate the model.


In [16]:
# Define a custom dense layer with 128 units
class CustomDenseLayer(Layer):
    def __init__(self, units=128):
        # Initialize the base Layer class and set the number of units
        super(CustomDenseLayer, self).__init__()
        self.units = units

    def build(self, input_shape):
        # Define trainable weights (w) and biases (b) for the layer
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),  # Shape: (input features, units)
            initializer='random_normal',         # Initialize weights with a random normal distribution
            trainable=True                       # Make weights trainable
        )
        self.b = self.add_weight(
            shape=(self.units,),                 # Shape: (units,)
            initializer='zeros',                # Initialize biases to zeros
            trainable=True                       # Make biases trainable
        )

    def call(self, inputs):
        # Perform the forward pass: inputs * weights + biases, followed by ReLU activation
        return tf.nn.relu(tf.matmul(inputs, self.w) + self.b)

# Integrate the new custom layer into a sequential model
model = Sequential([
    CustomDenseLayer(128),  # Custom dense layer with 128 units
    CustomDenseLayer(10)    # Custom dense layer with 10 units for multi-class logits
])

# Recompile the model
# - Use Adam optimizer for efficient gradient updates
# - Use categorical cross-entropy for multi-class classification
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Train the model again with the randomly generated dataset
# - epochs=10: Train the model for 10 full passes over the dataset
# - batch_size=32: Divide the data into mini-batches of 32 samples each for training
model.fit(x_train, y_train, epochs=10, batch_size=32)


Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 8.5269  
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 6.6690
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 6.5338
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.2496
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.5194
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.2263
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.1274
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.5202
Epoch 9/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.2905
Epoch 10/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6.1363


<details>
    <summary>Click here for Solution</summary>

```python

# Define a custom layer with 128 units
class CustomDenseLayer(Layer):
    def __init__(self, units=128):
        super(CustomDenseLayer, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='zeros',
                                 trainable=True)

    def call(self, inputs):
        return tf.nn.relu(tf.matmul(inputs, self.w) + self.b)

# Integrate the new custom layer into a model
model = Sequential([
    CustomDenseLayer(128),
    CustomDenseLayer(10)
])

# Recompile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Train the model again
model.fit(x_train, y_train, epochs=10, batch_size=32)
 ```   

</details>


### Summary

By completing these exercises, students will:

1. Visualize the architecture of their custom Keras model.
2. Understand the impact of adding Dropout layers to prevent overfitting.
3. Experiment with different configurations of the custom dense layer to observe performance changes.


## Conclusion 

Congratulations! You have successfully created and trained a custom layer in Keras. This lab exercise demonstrated how to extend Keras’s capabilities by creating custom layers and integrating them into a model. 


In [ ]:
Copyright © IBM Corporation. All rights reserved.